In [ ]:
%load_ext autoreload
%autoreload 2

import xarray as xr
import numpy as np
from dask.distributed import Client
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from scipy import stats
import xesmf as xe
from scipy.stats import linregress
import os

# Lets import our functions. There are no tests here. Stored separately so as to not make a mess.
# Make sure in right dir first

os.chdir('/g/data/xv83/np1383/CAFE60_eqpac/')
from a_carbon_math import carbon_flux
from a_model_cutting_functions import CAFE60_eqpac_cutter, cut_regrid_reynolds_sst,cut_process_sst_obs_trends,proc_landschutzer,process_co2_land_trends

In [ ]:
# Set up the remote dask cluster
from dask.distributed import Client,Scheduler
from dask_jobqueue import SLURMCluster
cluster = SLURMCluster(cores=2,memory="8GB")
client = Client(cluster)
#cluster.scale(cores=4)
cluster.adapt(minimum=2, maximum=6)

In [ ]:
def cut_regrid_sst():
    sst_obs=xr.open_dataset('/scratch1/pit071/carbon_data_ch2/sst.mnmean.nc')
    sst_obs=sst_obs.sst.sel(lat=slice(20,-20),lon=slice(120,290),time=slice('1982-01-01','2019-12-01'))
    regridder = xe.Regridder(sst_obs, sst_cafe, 'bilinear',reuse_weights=True)
    sst_obs_regrid=regridder(sst_obs)
    sst_obs_regrid.to_netcdf('/scratch1/pit071/CAFE60/sst.mnmean.regrid.nc')

In [ ]:
#Cut out global and eqpac reynolds SST and calculate trends for them. Saved in /scratch1/pit071/CAFE60/processed/obs/
cut_regrid_reynolds_sst(True,True) #Cut eqpac SST,force (If force=True will delete any existing file and resave)
cut_regrid_reynolds_sst(False,True) #Cut global SST and force save 
cut_process_sst_obs_trends(True) #Force trends for both global and eqpac. Should do both eqpac and global.  

#Basically the same as above but for landshutzer.
#proc_landschutzer(False,True) 
proc_landschutzer(True,True,True)
process_co2_land_trends(True)
proc_landschutzer(False,True,True) #Do old landshutzer as well